# Pose Estimation Related to Emotions

In [1]:
import cv2
from IPython.display import Image
import pandas as pd
from tools.segment import Segment
from tools.extractor import Extractor
from ultralytics import YOLO

/Users/deniskrylov/Developer/PosEmotion/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Data Preparation
Details about data preparation.

In [2]:
df = pd.read_csv("assets/annotations/annotations.csv")
df.head()

,Video Tag,Clip Id,Labels,Frame Number,X,Y,Width,Height,Person Id
0,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19532,41.965200,4.873195,44.216991,94.802684,0
1,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19538,41.564836,4.874640,44.216991,94.802684,0
2,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19544,41.164472,4.876086,44.216991,94.802684,0
3,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19550,40.764108,4.877532,44.216991,94.802684,0
4,aJKL0ahn1Dk,1,"[['Happy'], ['Happy'], ['Happy']]",19556,39.646728,5.014136,44.216991,94.802684,0


Extract all frames from videos and create segments that represent a unique person.

In [3]:
extractor = Extractor(
    "assets/annotations/annotations.csv",
    "assets/videos",
    "assets/frames"
)

# extractor.extract_frames()

NUM = 1
segments = extractor.segmentate()
segments[NUM].labels_to_probs()
segments[NUM].group

,Video Tag,Clip Id,Frame Number,Person Id,Happy,Sad,Fear,Neutral,Surprise,Disgust,Anger
28,aJKL0ahn1Dk,1,19700,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
29,aJKL0ahn1Dk,1,19706,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
30,aJKL0ahn1Dk,1,19712,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
31,aJKL0ahn1Dk,1,19718,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32,aJKL0ahn1Dk,1,19724,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
33,aJKL0ahn1Dk,1,19730,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34,aJKL0ahn1Dk,1,19736,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
35,aJKL0ahn1Dk,1,19742,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
36,aJKL0ahn1Dk,1,19748,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
37,aJKL0ahn1Dk,1,19754,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
segments[NUM].movement_speed()
segments[NUM].group

,Video Tag,Clip Id,Frame Number,Person Id,Happy,Sad,Fear,Neutral,Surprise,Disgust,Anger,Speed (seconds)
28,aJKL0ahn1Dk,1,19700,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
29,aJKL0ahn1Dk,1,19706,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
30,aJKL0ahn1Dk,1,19712,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
31,aJKL0ahn1Dk,1,19718,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
32,aJKL0ahn1Dk,1,19724,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
33,aJKL0ahn1Dk,1,19730,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
34,aJKL0ahn1Dk,1,19736,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
35,aJKL0ahn1Dk,1,19742,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
36,aJKL0ahn1Dk,1,19748,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1
37,aJKL0ahn1Dk,1,19754,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1


## Pose Analysis

### Pose Extraction
Details about pose extraction.

#### YOLO-Pose
Extracting poses from all frames using YOLO-Pose, and adding all data about extracted labels into new CSV file.

In [6]:
from tools.keypoints import Keypoints


model = YOLO('/Users/deniskrylov/Developer/PosEmotion/models/yolo-pose.pt')
img = cv2.imread('/Users/deniskrylov/Developer/PosEmotion/assets/frames/aJKL0ahn1Dk_19532.jpg')
results = model('/Users/deniskrylov/Developer/PosEmotion/assets/frames/aJKL0ahn1Dk_19532.jpg')[0]
keypoints = []

for result in results:
    for _, keypoint in enumerate(result.keypoints.xy.numpy().tolist()[0]):
        keypoints.append(keypoint)

k = Keypoints(image=img, keys=keypoints)
k.draw()


image 1/1 /Users/deniskrylov/Developer/PosEmotion/assets/frames/aJKL0ahn1Dk_19532.jpg: 768x1280 1 person, 1093.0ms
Speed: 2.6ms preprocess, 1093.0ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 1280)


Take a random row and draw the results.

#### OpenPose

#### DeepPose

### Pose Analysis as Skeletons

### Pose Analysis as Trajectories

### Cluster Evaluation

#### Single Skeletons

#### Trajectories

## Results